In [1]:
from env import user, password, host 
from acquire import acquire_data

import pandas as pd
import os.path

The data is stored on a sql database. We'll use an env.py with the credintials needed to pull the data from sql. We'll pull the data into a pandas dataframe, which will be the primary datastructure for housing our data.

# Getting the Data from SQL

In [3]:
query = """
SELECT  bedroomcnt, 
		bathroomcnt,
        calculatedfinishedsquarefeet,
		taxvaluedollarcnt,
        taxamount,
		fips
FROM properties_2017 as p
JOIN predictions_2017 as pr USING(`parcelid`)
WHERE propertylandusetypeid = 261 and (transactiondate >= '2017-05-01' AND transactiondate <= '2017-06-31')
"""

zillow_url = f'mysql+pymysql://{user}:{password}@{host}/zillow'

df = pd.read_sql(query, zillow_url)

In [4]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,fips
0,3.0,2.0,1458.0,136104.0,2319.90,6037.0
1,2.0,1.0,1421.0,35606.0,543.69,6037.0
2,4.0,3.0,2541.0,880456.0,9819.72,6059.0
3,3.0,2.0,1650.0,614000.0,7673.19,6037.0
4,2.0,1.0,693.0,274237.0,3267.47,6037.0


# Dealing with Counties

### Importing an external dataset that deals with FIPS, which can tell us which county the residence is in

In [ ]:
fips = pd.read_table('FIPS.txt')

In [ ]:
fips.head()

# Merging the Data

In [ ]:
df = pd.merge(left=df, right=fips, left_on='fips', right_on='FIPS')

In [ ]:
df.head()

# Generating a CSV to Reduce Stress on SQL Database

In [ ]:
if os.path.isfile('zillow.csv'):
    print ("CSV already generated.")
else:
    df.to_csv('zillow.csv')
    print('Generated csv within zillow-project directory')

# Testing the Module

In [ ]:
df = acquire_data()

In [ ]:
df.head()